<a href="https://colab.research.google.com/github/mnaveed8167/Keras-Tuner/blob/main/CNN_Optimization_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN *model* and optimization using keras tuner

In [2]:
 #!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.9 MB/s 
     |████████████████████████████████| 1.6 MB 37.6 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.8.2


In [4]:
fashio_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images, train_labels),(test_images, test_labels) = fashio_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
# Scaling down images
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.1 MB/s 


In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [14]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials= 5, directory = 'output', project_name = 'Mnist Fashion')

In [15]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.9078333377838135

Best val_accuracy So Far: 0.9104999899864197
Total elapsed time: 00h 02m 30s


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        96048     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 64)                1486912   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,584,410
Trainable params: 1,584,410
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images,train_labels,epochs =10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1412 - accuracy: 0.9472 - val_loss: 0.2701 - val_accuracy: 0.9115
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1007 - accuracy: 0.9628 - val_loss: 0.2883 - val_accuracy: 0.9110
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0751 - accuracy: 0.9721 - val_loss: 0.3085 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0547 - accuracy: 0.9799 - val_loss: 0.3588 - val_accuracy: 0.9093
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0407 - accuracy: 0.9853 - val_loss: 0.4064 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0351 - accuracy: 0.9877 - val_loss: 0.4468 - val_accuracy: 0.9122
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0283 - accuracy: 0.9896 - val_loss: 0.4882 - val_accuracy